In [1]:
import sys
import os
from dotenv import load_dotenv

load_dotenv()
work_dir = os.getenv('WORK_DIR')

sys.path.append(work_dir)

In [2]:
from src.db_connection import get_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy import inspect
from src.model import Candidates
from sqlalchemy.exc import SQLAlchemyError

In [3]:
engine = get_engine()

Successfully connected to the database workshop1!


In [4]:
Session = sessionmaker(bind=engine)
session = Session()

In [12]:
try:
    inspector = inspect(engine)
    
    if inspector.has_table('Candidates'):
        Candidates.__table__.drop(engine)
    
    Candidates.__table__.create(engine)
    print("Table creation was successful.")
    
except SQLAlchemyError as error:
    print(f"An error occurred while creating the table: {error}")
    
finally:
    engine.dispose()

Table creation was successful.
